In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import pandas as pd
import cv2
import os
import numpy as np
csv_path = '/home/d-zhukov/Projects/odometry/experiments/kitti_4_6/bovw_22/train/00/frame_history.csv'
dataset_root = '/datasets/odometry/KITTI_odometry_2012/'

In [ ]:
df = pd.read_csv(csv_path)
df.head()

In [ ]:
def visualize(df, only_loops, difference_threshold):
    """
    
    :param df: Dataframe
    :param only_loops: If True visualizes only loops
    :param difference_threshold: Maximum difference between frames indexes that considered as non loop 
    :return: 
    """
    frame_total = df.to_index.iloc[-1]
    frame_counter = 1
    
    while frame_counter <=  frame_total:  
        matches = df[df['to_index'] == frame_counter]
        frame_counter += 1
        
        if len(matches) > 0:
            
            if only_loops:
                max_difference = 0
                for k in range(len(matches)):
                    difference = matches['to_index'].iloc[k] - matches['from_index'].iloc[k]
                    if difference > max_difference:
                        max_difference = difference
                
                if max_difference < difference_threshold:
                    continue
            
            print(matches[['to_index', 'from_index']])
            
            image = cv2.imread(os.path.join(dataset_root, matches.to_path.iloc[0]))
            fig, axes = plt.subplots(2, 2, figsize=(40, 10))
            axes[0][0].imshow(image / 255)
            for k in range(3):
                if k < len(matches):
                    image = cv2.imread(os.path.join(dataset_root, matches.from_path.iloc[k]))
                else:
                    image = np.zeros_like(image)
                axes[(k + 1) // 2][(k + 1) % 2].imshow(image / 255)
            plt.show()

In [ ]:
is_keyframe = ~np.isnan(df['to_db_index'])
df = df[is_keyframe]

visualize(df, only_loops=True, difference_threshold=100)